Problems
Wikipedia search ending different
repitition n excel
Update vs New file

In [1]:
##Packages
import requests
import unicodedata
from xlrd import open_workbook
import string
import re
import numpy
from xlsxwriter import Workbook

In [2]:
def remSite(x):
    return x.split('/')[-1].replace('_',' ')

In [3]:
def makeneo(x):
    '''
    makes input into neo4j compatible code'''
    a= re.sub("[\(\[].*?[\)\]]", "", rem_uni(x)).translate(None, " ").translate(None, string.punctuation)
    return a

In [4]:
#Define keys to look for
wantedkeys=['http://dbpedia.org/ontology/birthDate',
            'http://dbpedia.org/ontology/networth',
            'http://dbpedia.org/ontology/residence',
            'http://dbpedia.org/ontology/salary',
            'http://dbpedia.org/ontology/spouse',
            'http://dbpedia.org/ontology/nationality',
            'http://dbpedia.org/property/nationality',
            'http://xmlns.com/foaf/0.1/gender',
            'http://dbpedia.org/ontology/almaMater',
            'http://dbpedia.org/property/almaMater',
            'http://dbpedia.org/property/knownFor',
            'http://dbpedia.org/ontology/education',
            'http://dbpedia.org/ontology/occupation',
            'http://dbpedia.org/property/title',
            'http://dbpedia.org/ontology/board',
            'http://dbpedia.org/property/spouse',
            'http://dbpedia.org/ontology/industry',
            'http://dbpedia.org/ontology/product',
            'http://dbpedia.org/property/areaServed',
            'http://dbpedia.org/property/hqLocationCity',
            'http://dbpedia.org/property/hqLocationCountry',
            'http://dbpedia.org/ontology/assets',
            'http://dbpedia.org/ontology/foundingYear',
            'http://dbpedia.org/property/products',
            'http://dbpedia.org/ontology/locationCity',
            'http://dbpedia.org/ontology/locationCountry',
            'http://dbpedia.org/ontology/regionServed',
            'http://dbpedia.org/ontology/location',
            'http://dbpedia.org/ontology/netIncome',
            'http://dbpedia.org/ontology/city'
           ]
fieldnames=['name','Label','Name','InDB','Company']
keys=[]
for key in wantedkeys:
    if remSite(key) not in keys:
        keys+=[remSite(key)]
fieldnames+=sorted(keys)
values=['NA']*len(fieldnames)

In [6]:
def rem_uni(u):
    '''
    This function removes unicode & returns a string
    '''
    if type(u)== unicode:
        s=unicodedata.normalize('NFKD',u).encode('ascii','ignore')
    else:
        s=str(u)
    return s

In [7]:
def retrieve(name):
    '''
    Pulls data from dbpedia
    '''
    name=name.replace(' ','_')
    req='http://dbpedia.org/data/'+name+'.json'
    dat='http://dbpedia.org/resource/'+name
    data = requests.get(req).json()
    q_json = data[dat]
    return q_json

In [8]:
def persondict(data, wantedkeys=wantedkeys,fieldnames=fieldnames):
    '''
    This function creates dictionary on person
    '''
    dic = {field:'NA' for field in fieldnames}
    other=[]
    others=[]
    for key in sorted(data):
        val=''
        o=[]
        if key in wantedkeys:
            k=remSite(rem_uni(key))
            for item in data[key]:
                val+=' '+remSite(rem_uni(item['value']))
                o+=[remSite(rem_uni(item['value']))]
                if "http://" in rem_uni(item['value']):
                    others+=o
                    if [k,o] not in other:
                        other+=[[k,o]]
                dic[k]=val[1:]
    return dic,other, others

In [9]:
def main(query, wantedkeys=wantedkeys):
    r=retrieve(query)
    d= persondict(r)
    others=d[2]
    other=d[1]
    d=d[0]
    d['Name']=query
    d['InDB']='Y'
    return d,other, others

In [10]:
info=open_workbook('C:\Users\cacerb1\Documents\EDO-Intern-Project\Brittany\Directors\DBPedia\TopTen.xlsx')
info=info.sheets()[0]

In [11]:
nodes=[]
companies=[]
other=[]
others=[]
for r in range(1,info.nrows):
    node=[]
    query=rem_uni(info.cell(r,3).value)
    company=rem_uni(info.cell(r,2).value)
    name=makeneo(info.cell(r,0).value)
    label=rem_uni(info.cell(r,1).value)
    companies+=[company]
    try:
        node= main(query)
        other+=[[name]+[[['Company',company]]+node[1]]]
        others += node[2]
        node=node[0]
        node['name']=name
        node['Company']=company
        node['Label']=label
    except KeyError:
        node= {field:'NA' for field in fieldnames}
        node['Name']=query
        node['InDB']='N'
        node['Company']=company
        node['name']=name
        node['Label']=label
        other+=[[name]+[[['Company',company]]]]
    nodes+=[node]

In [12]:
others_sorted=list(numpy.unique(others))
testd=[]
test1=[]
test2=[]
desc=[]
for o in others_sorted:
    if o[-1] =='1':
        desc+=[o]
    else:
        try:
            x=main(o)
            d=x[0]
            t1=x[1]
            t2=x[2]
            name=makeneo(o)
            d['name']=name
            d['Company']='NA'
            d['Label']='Rel'
            nodes+=[d]
            test1+=[t1]
            test2+=[t2]
        except (KeyError, ValueError):
            node= {field:'NA' for field in fieldnames}
            node['InDB']='N'
            node['Name']=o
            node['name']=makeneo(o)
            node['Label']='Rel'
            nodes+=[node]

In [13]:
wb=Workbook("TopTenAttributes.xlsx")
a=wb.add_worksheet("Attributes")
for header in fieldnames:
    col=fieldnames.index(header) # we are keeping order.
    a.write(0,col,header)
row=1
for node in nodes:
    for _key,_value in node.items():
        col=fieldnames.index(_key)
        a.write(row,col,_value)
    row+=1 #enter the next row

In [14]:
r=wb.add_worksheet("Relationships")
heads=['Node A','Rel','Node B']
for header in heads:
    col=heads.index(header) # we are keeping order.
    r.write(0,col,header)
row=1
for item in other:
    for rel in item[1]:
        if type(rel[1])==list:
            for t in rel[1]:
                if 'NA' not in t:
                    if t[-1]!='1':
                        r.write(row,0,makeneo(item[0]))
                        r.write(row,1,rel[0])
                        r.write(row,2,makeneo(t))
                        row+=1 #enter the next row 
        elif 'NA' not in rel[1]:
            if rel[1][-1] != '1':
                a=rel[1]
                r.write(row,0,makeneo(item[0]))
                r.write(row,1,rel[0])
                r.write(row,2,makeneo(rel[1]))
                row+=1 #enter the next row 
wb.close()